## Importing Libraries

In [ ]:
import os
import wandb
import shutil
import polars as pl
from datasets import Dataset
from pathlib import Path
import numpy as np
import pandas as pd
import torch  # base
import torch.nn.functional as F
import json
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from datetime import datetime
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

## Setting Cuda

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA (NVIDIA GPU)")
else:
    device = torch.device("cpu")
    print("Using CPU")

print(f"Device: {device}")

# 🤖 ChatBot Arena Dataset Overview

This dataset comes from the **ChatBot Arena**, where judges evaluate responses from two competing large language models (LLMs) given the same prompt.

## 📊 Dataset Statistics
- **📦 Train Size:** 55,000 interactions
- **🧪 Test Size:** ~25,000 interactions  
- **📁 Files Included:** `train.csv`, `test.csv`

## 📑 `train.csv` Columns

- `id`: Unique row identifier
- `model_a` / `model_b`: Model names (only in train)
- `prompt`: The input prompt shown to both models
- `response_a` / `response_b`: Respective model responses
- `winner_model_a` / `winner_model_b` / `winner_tie`: One-hot encoded ground truth indicating which model won or if it was a tie

## 📑 `test.csv` Columns

- `id`: Unique row identifier
- `prompt`: The shared prompt
- `response_a` / `response_b`: Responses from the two anonymous models

## 🧠 Objective

The objective is to train a model to predict the more preferred response (*model_a*, *model_b*, or *tie*) based on the prompt and both responses.

In [ ]:
# Assuming the data files are in '../input/llm-classification-finetuning/'
# You'll need to check the actual file names and paths once you join the competition.

try:
    train_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
    test_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')
    # You might also have a sample submission file to guide the format
    sample_submission_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/sample_submission.csv')

    print("Data loaded successfully.")
    print(f"Train data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")
    print(f"Sample Submission data shape: {sample_submission_df.shape}")

except FileNotFoundError:
    print("Error: Data files not found. Please ensure the dataset is added to your Kaggle Notebook.")
    print("Go to 'Add Data' on the right sidebar of your notebook and search for 'llm-classification-finetuning'.")
    exit() # Exit if data isn't found to prevent further errors

# Display the first few rows of the dataframes to understand their structure
print("\nTrain DataFrame Head:")
print(train_df.head())
print("\nTest DataFrame Head:")
print(test_df.head())
print("\nSample Submission DataFrame Head:")
print(sample_submission_df.head())

## EDA

In [ ]:
print("\n=== MISSING VALUES ===")
print(train_df.isnull().sum())

print("\n=== UNIQUE VALUES IN KEY COLUMNS ===")
print(f"Unique model_a: {train_df['model_a'].nunique()}")
print(f"Unique model_b: {train_df['model_b'].nunique()}")
print(f"Sample models: {train_df['model_a'].unique()[:5]}")

print("\n=== TARGET DISTRIBUTION ===")
print(f"winner_model_a = 1: {train_df['winner_model_a'].sum()}")
print(f"winner_model_b = 1: {train_df['winner_model_b'].sum()}")
print(f"winner_tie = 1: {train_df['winner_tie'].sum()}")

print("\n=== DATA QUALITY CHECKS ===")
print(f"Total rows: {len(train_df)}")
print(f"Rows where exactly one winner is 1:")
valid_rows = ((train_df['winner_model_a'] + train_df['winner_model_b'] + train_df['winner_tie']) == 1).sum()
print(f"Valid rows: {valid_rows}")
print(f"Invalid rows: {len(train_df) - valid_rows}")


In [ ]:
print("=== CONVERTING STRING LISTS TO ACTUAL LISTS ===")
# Check if prompt/responses are string representations of lists
print("Sample prompt type:", type(train_df['prompt'].iloc[0]))
print("Sample prompt value:", train_df['prompt'].iloc[0])

def safe_eval(x):
    """Convert string representation of list to actual list"""
    if isinstance(x, str):
        try:
            # Try to evaluate as Python literal (safe)
            result = ast.literal_eval(x)
            return result if isinstance(result, list) else [result]
        except:
            # If it fails, treat as regular string
            return [x]
    return x

# Apply to prompt and response columns
train_df['prompt_processed'] = train_df['prompt'].apply(safe_eval)
train_df['response_a_processed'] = train_df['response_a'].apply(safe_eval)
train_df['response_b_processed'] = train_df['response_b'].apply(safe_eval)

print("\n=== CHECKING PROCESSED DATA ===")
print("Sample processed prompt:", train_df['prompt_processed'].iloc[0])
print("Type:", type(train_df['prompt_processed'].iloc[0]))

# Check if we have multiple prompts/responses per row
print("\n=== CHECKING FOR MULTIPLE PROMPTS/RESPONSES ===")
prompt_lengths = train_df['prompt_processed'].apply(len)
response_a_lengths = train_df['response_a_processed'].apply(len)
response_b_lengths = train_df['response_b_processed'].apply(len)

print(f"Prompt lengths: min={prompt_lengths.min()}, max={prompt_lengths.max()}, mean={prompt_lengths.mean():.2f}")
print(f"Response A lengths: min={response_a_lengths.min()}, max={response_a_lengths.max()}, mean={response_a_lengths.mean():.2f}")
print(f"Response B lengths: min={response_b_lengths.min()}, max={response_b_lengths.max()}, mean={response_b_lengths.mean():.2f}")

# Extract first prompt/response from each list
def extract_first(x):
    """Extract first element if it's a list, otherwise return as string"""
    if isinstance(x, list):
        return x[0] if len(x) > 0 else ""
    return str(x)

train_df['prompt_text'] = train_df['prompt_processed'].apply(extract_first)
train_df['response_a_text'] = train_df['response_a_processed'].apply(extract_first)
train_df['response_b_text'] = train_df['response_b_processed'].apply(extract_first)

print("\n=== FINAL TEXT DATA ===")
print(f"Sample prompt text: {train_df['prompt_text'].iloc[0][:100]}...")
print(f"Sample response A text: {train_df['response_a_text'].iloc[0][:100]}...")
print(f"Sample response B text: {train_df['response_b_text'].iloc[0][:100]}...")

# Check text lengths
print(f"\nPrompt text lengths: mean={train_df['prompt_text'].str.len().mean():.0f}, max={train_df['prompt_text'].str.len().max()}")
print(f"Response A text lengths: mean={train_df['response_a_text'].str.len().mean():.0f}, max={train_df['response_a_text'].str.len().max()}")
print(f"Response B text lengths: mean={train_df['response_b_text'].str.len().mean():.0f}, max={train_df['response_b_text'].str.len().max()}")

## CREATING TARGET LABELS

In [ ]:
print("=== CREATING TARGET LABELS ===")

# Check current winner columns
print("Current winner columns:")
print(f"winner_model_a: {train_df['winner_model_a'].value_counts()}")
print(f"winner_model_b: {train_df['winner_model_b'].value_counts()}")
print(f"winner_tie: {train_df['winner_tie'].value_counts()}")

# Check if each row has exactly one winner
print("\n=== CHECKING DATA VALIDITY ===")
winner_sum = train_df['winner_model_a'] + train_df['winner_model_b'] + train_df['winner_tie']
print(f"Rows with exactly one winner: {(winner_sum == 1).sum()}")
print(f"Rows with no winner: {(winner_sum == 0).sum()}")
print(f"Rows with multiple winners: {(winner_sum > 1).sum()}")

# Create single target label
def get_winner_label(row):
    """Convert winner columns to single label"""
    if row['winner_model_a'] == 1:
        return 0  # Model A wins
    elif row['winner_model_b'] == 1:
        return 1  # Model B wins
    elif row['winner_tie'] == 1:
        return 2  # Tie
    else:
        return -1  # Invalid/missing label

train_df['label'] = train_df.apply(get_winner_label, axis=1)

print("\n=== LABEL DISTRIBUTION ===")
print(f"Label 0 (Model A wins): {(train_df['label'] == 0).sum()}")
print(f"Label 1 (Model B wins): {(train_df['label'] == 1).sum()}")
print(f"Label 2 (Tie): {(train_df['label'] == 2).sum()}")
print(f"Label -1 (Invalid): {(train_df['label'] == -1).sum()}")

# Remove invalid labels
print(f"\nRows before cleaning: {len(train_df)}")
train_df_clean = train_df[train_df['label'] != -1].copy()
print(f"Rows after cleaning: {len(train_df_clean)}")

# Final label distribution
labels = train_df_clean['label'].values
print(f"\n=== FINAL LABEL DISTRIBUTION ===")
print(f"Model A wins: {np.sum(labels == 0)} ({np.mean(labels == 0)*100:.1f}%)")
print(f"Model B wins: {np.sum(labels == 1)} ({np.mean(labels == 1)*100:.1f}%)")
print(f"Tie: {np.sum(labels == 2)} ({np.mean(labels == 2)*100:.1f}%)")

# Show some examples
print(f"\n=== SAMPLE LABELED DATA ===")
for i in range(3):
    row = train_df_clean.iloc[i]
    label_text = ['Model A wins', 'Model B wins', 'Tie'][row['label']]
    print(f"\nExample {i+1}:")
    print(f"Prompt: {row['prompt_text'][:80]}...")
    print(f"Response A: {row['response_a_text'][:80]}...")
    print(f"Response B: {row['response_b_text'][:80]}...")
    print(f"Label: {row['label']} ({label_text})")

# Update train_df to clean version
train_df = train_df_clean
print(f"\nDataset ready for training with {len(train_df)} samples!")

## CREATING TRAINING FEATURES

In [ ]:
# Step 4: Create Training Features

print("=== CREATING TRAINING FEATURES ===")

# Method 1: Simple concatenation for transformer models
def create_model_input(row):
    """Create input text for transformer models"""
    return f"Prompt: {row['prompt_text']} Response A: {row['response_a_text']} Response B: {row['response_b_text']}"

train_df['model_input'] = train_df.apply(create_model_input, axis=1)

print("=== FEATURE STATISTICS ===")
input_lengths = train_df['model_input'].str.len()
print(f"Input text lengths:")
print(f"  Mean: {input_lengths.mean():.0f} characters")
print(f"  Min: {input_lengths.min()} characters")
print(f"  Max: {input_lengths.max()} characters")
print(f"  Median: {input_lengths.median():.0f} characters")

# Check for very long inputs (might need truncation)
long_inputs = (input_lengths > 4000).sum()
print(f"  Inputs > 4000 chars: {long_inputs} ({long_inputs/len(train_df)*100:.1f}%)")

print("\n=== SAMPLE TRAINING FEATURES ===")
for i in range(3):
    row = train_df.iloc[i]
    label_text = ['Model A wins', 'Model B wins', 'Tie'][row['label']]
    print(f"\nExample {i+1}:")
    print(f"Input: {row['model_input'][:200]}...")
    print(f"Label: {row['label']} ({label_text})")

# Create X (features) and y (labels) arrays
X = train_df['model_input'].values
y = train_df['label'].values

print(f"\n=== FINAL TRAINING DATA ===")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X sample: {X[0][:100]}...")
print(f"y sample: {y[:10]}")

# Check for any missing values
print(f"\nMissing values in X: {sum(1 for x in X if pd.isna(x) or x == '')}")
print(f"Missing values in y: {sum(1 for label in y if pd.isna(label))}")

print("\n=== READY FOR TRAIN/VALIDATION SPLIT! ===")

## SPLITTING DATA

In [ ]:
print("=== SPLITTING DATA ===")
print(f"Total samples: {len(X)}")
print(f"Label distribution before split:")
print(f"  Model A wins: {np.sum(y == 0)} ({np.mean(y == 0)*100:.1f}%)")
print(f"  Model B wins: {np.sum(y == 1)} ({np.mean(y == 1)*100:.1f}%)")
print(f"  Tie: {np.sum(y == 2)} ({np.mean(y == 2)*100:.1f}%)")

# Split data with stratification to maintain label distribution
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2,           # 20% for validation
    stratify=y,              # Keep same label distribution in both sets
    random_state=42          # For reproducibility
)

print(f"\n=== SPLIT RESULTS ===")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")

print(f"\nTraining set label distribution:")
print(f"  Model A wins: {np.sum(y_train == 0)} ({np.mean(y_train == 0)*100:.1f}%)")
print(f"  Model B wins: {np.sum(y_train == 1)} ({np.mean(y_train == 1)*100:.1f}%)")
print(f"  Tie: {np.sum(y_train == 2)} ({np.mean(y_train == 2)*100:.1f}%)")

print(f"\nValidation set label distribution:")
print(f"  Model A wins: {np.sum(y_val == 0)} ({np.mean(y_val == 0)*100:.1f}%)")
print(f"  Model B wins: {np.sum(y_val == 1)} ({np.mean(y_val == 1)*100:.1f}%)")
print(f"  Tie: {np.sum(y_val == 2)} ({np.mean(y_val == 2)*100:.1f}%)")

print(f"\n=== DATA READY FOR MODEL TRAINING! ===")
print("Variables created:")
print("  X_train: Training features")
print("  y_train: Training labels")
print("  X_val: Validation features")
print("  y_val: Validation labels")

## Using TF-IDF + Logistic Regression

In [ ]:
# print("=== TRAINING BASELINE MODEL ===")
# print("Using TF-IDF + Logistic Regression")

# # Step 1: Convert text to numerical features using TF-IDF
# print("\n1. Converting text to TF-IDF features...")
# vectorizer = TfidfVectorizer(
#     max_features=10000,      # Use top 10k words
#     stop_words='english',    # Remove common words
#     ngram_range=(1, 2),      # Use 1-word and 2-word phrases
#     max_df=0.95,             # Ignore words in >95% of docs
#     min_df=2                 # Ignore words in <2 docs
# )

# # Fit on training data and transform both sets
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_val_tfidf = vectorizer.transform(X_val)

# print(f"TF-IDF feature shape: {X_train_tfidf.shape}")

# # Step 2: Train Logistic Regression
# print("\n2. Training Logistic Regression...")
# model = LogisticRegression(
#     max_iter=1000,
#     random_state=42,
#     class_weight='balanced'  # Handle class imbalance
# )

# model.fit(X_train_tfidf, y_train)
# print("Model trained!")

# # Step 3: Make predictions
# print("\n3. Making predictions...")
# y_train_pred = model.predict(X_train_tfidf)
# y_val_pred = model.predict(X_val_tfidf)

# # Step 4: Evaluate performance
# print("\n=== MODEL PERFORMANCE ===")
# train_accuracy = accuracy_score(y_train, y_train_pred)
# val_accuracy = accuracy_score(y_val, y_val_pred)

# print(f"Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
# print(f"Validation Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")

# # Check if overfitting
# if train_accuracy - val_accuracy > 0.05:
#     print("⚠️  Possible overfitting detected!")
# else:
#     print("✅ Good generalization!")

# # Detailed performance report
# print(f"\n=== DETAILED VALIDATION RESULTS ===")
# print(classification_report(y_val, y_val_pred, 
#                           target_names=['Model A wins', 'Model B wins', 'Tie']))

# # Confusion matrix
# print(f"\n=== CONFUSION MATRIX ===")
# cm = confusion_matrix(y_val, y_val_pred)
# print("Rows: True labels, Columns: Predicted labels")
# print("        A_wins  B_wins  Tie")
# for i, row_label in enumerate(['A_wins', 'B_wins', 'Tie']):
#     print(f"{row_label:>7} {cm[i]}")

# # Show some prediction examples
# print(f"\n=== PREDICTION EXAMPLES ===")
# for i in range(5):
#     true_label = ['Model A wins', 'Model B wins', 'Tie'][y_val[i]]
#     pred_label = ['Model A wins', 'Model B wins', 'Tie'][y_val_pred[i]]
#     correct = "✅" if y_val[i] == y_val_pred[i] else "❌"
    
#     print(f"\nExample {i+1}: {correct}")
#     print(f"Input: {X_val[i][:100]}...")
#     print(f"True: {true_label}")
#     print(f"Predicted: {pred_label}")

# print(f"\n=== BASELINE RESULTS SUMMARY ===")
# print(f"Validation Accuracy: {val_accuracy:.4f}")
# print("This is your baseline to beat with better models!")

#### Accuracy is very less on regression cause for this type of dataset we need to capture the sematic/contextual meaning of data for that we need to use transformers model like BERT

## Preparing Dataset for Finetuning

In [ ]:
model_path = "/kaggle/input/bert/transformers/default/1/bert-base-uncased-local"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)

In [ ]:
# Encode labels
label_names = ['Model A wins', 'Model B wins', 'Tie']
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_val_enc = label_encoder.transform(y_val)

# Tokenize data
def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

def clean_text(text):
    return text.encode("utf-8", "ignore").decode("utf-8", "ignore")
    
X_train_cleaned = [clean_text(t) for t in X_train]
X_val_cleaned = [clean_text(t) for t in X_val]


# Wrap into HuggingFace Dataset format
train_dataset = Dataset.from_dict({'text': X_train_cleaned, 'label': y_train_enc})
val_dataset = Dataset.from_dict({'text': X_val_cleaned, 'label': y_val_enc})

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


## Fine Tuning Using Bert

In [ ]:
# ✅ Paths & Configs
os.environ["WANDB_MODE"] = "disabled"
MODEL_DIR = "/kaggle/working/models"
timestamp = datetime.now().strftime("%Y%m%d-%H%M")
run_name = f"deberta-v3-large-{timestamp}"

# ✅ Training Arguments
training_args = TrainingArguments(
    output_dir=f"{MODEL_DIR}/results",
    run_name=run_name,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=10000,
    logging_first_step=True,
    eval_strategy="no",
    save_strategy="no",
    fp16=True,
    dataloader_num_workers=0,
    seed=42,
    report_to=[]
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {
        'accuracy': (p.predictions.argmax(axis=-1) == p.label_ids).mean()}
)

In [ ]:
trainer.train()

In [ ]:
MODEL_DIR = "./models"
final_model_path = f"{MODEL_DIR}/final"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

## Evaluation

In [ ]:
# results = trainer.evaluate()
# print(results)

# # Classification Repor
# preds = trainer.predict(val_dataset)
# y_pred = np.argmax(preds.predictions, axis=1)

# print(classification_report(y_val_enc, y_pred, target_names=label_names))


## Inference

In [ ]:
def load_model(model_path):
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer


model, tokenizer = load_model(final_model_path)
model.to(device)

text = "This is a test sentence"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs.to(device))
predictions = outputs.logits
predictions

In [ ]:
preds = F.softmax(predictions, dim=-1)
preds

In [ ]:
def apply_bert_fmt(df: pd.DataFrame) -> pd.DataFrame:
    df["text"] = df["prompt"] + " [SEP] " + df["response_a"] + " [SEP] " + df["response_b"]
    return df

In [ ]:
df_test = apply_bert_fmt(test_df)

In [ ]:
# Step 1: Tokenize texts
encodings = tokenizer(
    df_test["text"].tolist(),
    truncation=True,
    padding=True,
    return_tensors="pt"
)

# Step 2: Create a custom Dataset
class CustomTextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

test_dataset = CustomTextDataset(encodings)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Step 3: Run inference
all_probabilities = []

model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        inputs = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**inputs)
        probabilities = F.softmax(outputs.logits, dim=-1)
        all_probabilities.extend(probabilities.cpu().numpy())

final_probs = np.vstack(all_probabilities)


In [ ]:
final_probs

## Submission

In [ ]:
submission_df = df_test
submission_df["winner_model_a"] = final_probs[:, 0]
submission_df["winner_model_b"] = final_probs[:, 1]
submission_df["winner_tie"] = final_probs[:, 2]

submission_df = submission_df[["id", "winner_model_a", "winner_model_b", "winner_tie"]]
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)
submission_df.head()